<b><font color="red" size="6">ch14. 웹데이터 수집</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
- https://selenium-python.readthedocs.io/index.html
<br>`pip install selenium`

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
dv = webdriver.Chrome()
dv.get('http://python.org')
elem = dv.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')

In [3]:
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # enter키 효과

In [4]:
btn_elem = dv.find_element(By.CSS_SELECTOR, 'button#submit')
btn_elem.click()

In [5]:
result_list = dv.find_elements(By.CSS_SELECTOR, 'li > h3 > a')
for result in result_list:
    print("%s - %s" % (result.text, result.get_attribute('href')))

PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
PyCon Australia 2013 - https://www.python.org/events/python-events/57/
PyCon AU 2019 - https://www.python.org/events/python-events/776/
PyCon Australia 2014 - https://www.python.org/events/python-events/10/
PyCon Ireland 2012 - https://www.python.org/events/python-events/76/
PyCon Ireland 2016 - https://www.python.org/events/python-events/429/
PyCon Ireland 2022 - https://www.python.org/events/python-events/1320/
PyCon Australia 2014 - https://www.python.org/events/python-events/1447/
PyCon Ireland 2024 - https://www.python.org/events/python-events/1862/
PyCon APAC 2025 - https://www.python.org/events/python-events/1879/
PyCon AU 2018 - https://www.python.org/events/python-events/696/
PyCon APAC 2022 - https://www.python.org/events/python-events/1216/
PyCon PH 2024 - https://www.python.org/events/python-events/1661/
PyCon Irel

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(dv.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print('%s - %s' % (result.text, result.attrs['href']))

PSF PyCon Trademark Usage Policy - /psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - /events/python-events/378/
PyCon Australia 2013 - /events/python-events/57/
PyCon AU 2019 - /events/python-events/776/
PyCon Australia 2014 - /events/python-events/10/
PyCon Ireland 2012 - /events/python-events/76/
PyCon Ireland 2016 - /events/python-events/429/
PyCon Ireland 2022 - /events/python-events/1320/
PyCon Australia 2014 - /events/python-events/1447/
PyCon Ireland 2024 - /events/python-events/1862/
PyCon APAC 2025 - /events/python-events/1879/
PyCon AU 2018 - /events/python-events/696/
PyCon APAC 2022 - /events/python-events/1216/
PyCon PH 2024 - /events/python-events/1661/
PyCon Ireland 2023 - /events/python-events/1568/
PyCon PL 2014 - /events/python-events/191/
PyCon MY 2015 - /events/python-events/313/
PyCon Ireland 2015 - /events/python-events/333/
PyCon AU 2015 - /events/python-events/273/
PyCon Australia 2016 - /events/python-events/357/


In [7]:
from urllib.parse import urlparse
current_url = dv.current_url # 셀레니윰을 통해 접근한 현재 url
print('현재주소 :',current_url)
parse_url = urlparse(current_url)
print(parse_url,parse_url.scheme, parse_url.netloc)
domain = f'{parse_url.scheme}://{parse_url.netloc}'
domain = '{}://{}'.format(parse_url.scheme, parse_url.netloc)
print('도메인 :',domain)

현재주소 : https://www.python.org/search/?q=pycon&submit=
ParseResult(scheme='https', netloc='www.python.org', path='/search/', params='', query='q=pycon&submit=', fragment='') https www.python.org
도메인 : https://www.python.org


In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(dv.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print('%s - %s' % (result.text, domain+result.attrs['href']))

PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
PyCon Australia 2013 - https://www.python.org/events/python-events/57/
PyCon AU 2019 - https://www.python.org/events/python-events/776/
PyCon Australia 2014 - https://www.python.org/events/python-events/10/
PyCon Ireland 2012 - https://www.python.org/events/python-events/76/
PyCon Ireland 2016 - https://www.python.org/events/python-events/429/
PyCon Ireland 2022 - https://www.python.org/events/python-events/1320/
PyCon Australia 2014 - https://www.python.org/events/python-events/1447/
PyCon Ireland 2024 - https://www.python.org/events/python-events/1862/
PyCon APAC 2025 - https://www.python.org/events/python-events/1879/
PyCon AU 2018 - https://www.python.org/events/python-events/696/
PyCon APAC 2022 - https://www.python.org/events/python-events/1216/
PyCon PH 2024 - https://www.python.org/events/python-events/1661/
PyCon Irel

In [9]:
dv.close() # 브라우저 종료

# 2절. 동적웹크롤링 예제
## 2-1 다음 뉴스 검색

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
url = 'https://daum.net'
driver.get(url)
# driver.implicitly_wait(0.5) # 0.5초 요소를 찾을 때까지 대기시간
time.sleep(0.5) # 0.5초 대기
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는 ?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

검색할 단어는 ?비트코인


In [16]:
# 뉴스 탭 클릭
driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()

In [17]:
news_list = []
bodies = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong.tit-g.clamp-g')
bodies = driver.find_elements(By.CSS_SELECTOR, 'div[class="item-title"] > strong.tit-g.clamp-g')
# print(len(bodies))

for body in bodies:
    a = body.find_element(By.TAG_NAME, 'a')
    title = a.text
    link = a.get_attribute('href')
    news_list.append([title, link])
    print(title,link)

월가 차트분석 달인 "비트코인 85K 추락 가능성" http://v.daum.net/v/20241211104920355
'트럼프 효과 다했나'...비트코인 등 가상화폐 급락 http://v.daum.net/v/20241211232526095
새로운 머스크 수혜주 등장?…美대선 이후 주가 4배 껑충 http://v.daum.net/v/20241211175004287
가상화폐 ‘트럼프 효과’ 끝? 비트코인 10만 달러 밑으로 http://v.daum.net/v/20241211184115944
코인 전문가 오태민 "계엄 당일 우리나라만 폭락?..'리플'·'테더' 관심집중 http://v.daum.net/v/20241211191806736
구글 양자컴퓨터 출시에 비트코인 흔들..."영향없다" 반론도 http://v.daum.net/v/20241211135155454
"비트코인 투자하면 주가 올라" 설득했지만…MS 주주들은 '반대' http://v.daum.net/v/20241211133550900
숨 고르기 들어간 비트코인? http://v.daum.net/v/20241211142129680
비트코인 레버리지·공매도 버젓이… 당국은 `나 몰라라` http://v.daum.net/v/20241211150512600
트럼프 효과 끝?…비트코인 9만5천달러 하회 http://v.daum.net/v/20241211142219721


In [18]:
page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
next_page = page_nav.find_element(By.LINK_TEXT, '2')
next_page.click()

## 2-2 페이징 처리
- 다음 뉴스 페이징 처리

In [19]:
# 원하는 페이지 만큼 검색하여 검색 결과 데이터프레임으로 출력
driver = webdriver.Chrome()
time.sleep(1)

url = 'https://daum.net'
driver.get(url)
# driver.implicitly_wait(0.5) # 0.5초 요소를 찾을 때까지 대기시간
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는 ?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

# 뉴스 탭 클릭
driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()

news_list = []

page_numbers = int(input('몇 페이지 크롤링 할까요?'))
for page_number in range(1, page_numbers+1):
    
    bodies = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong.tit-g.clamp-g')
    bodies = driver.find_elements(By.CSS_SELECTOR, 'div[class="item-title"] > strong.tit-g.clamp-g')
    # print(len(bodies))
    print(f'----------------{page_number}page 크롤링 결과----------------')
    for body in bodies:
        a = body.find_element(By.TAG_NAME, 'a')
        title = a.text
        link = a.get_attribute('href')
        news_list.append([title, link])
        print(title,link)

    page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
    next_page = page_nav.find_element(By.LINK_TEXT, str(page_number+1))
    next_page.click()
    time.sleep(1)
driver.close()
import pandas as pd
display(pd.DataFrame(news_list, columns=['title','link']).head())


검색할 단어는 ?비트코인
몇 페이지 크롤링 할까요?3
----------------1page 크롤링 결과----------------
월가 차트분석 달인 "비트코인 85K 추락 가능성" http://v.daum.net/v/20241211104920355
'트럼프 효과 다했나'...비트코인 등 가상화폐 급락 http://v.daum.net/v/20241211232526095
새로운 머스크 수혜주 등장?…美대선 이후 주가 4배 껑충 http://v.daum.net/v/20241211175004287
가상화폐 ‘트럼프 효과’ 끝? 비트코인 10만 달러 밑으로 http://v.daum.net/v/20241211184115944
코인 전문가 오태민 "계엄 당일 우리나라만 폭락?..'리플'·'테더' 관심집중 http://v.daum.net/v/20241211191806736
구글 양자컴퓨터 출시에 비트코인 흔들..."영향없다" 반론도 http://v.daum.net/v/20241211135155454
"비트코인 투자하면 주가 올라" 설득했지만…MS 주주들은 '반대' http://v.daum.net/v/20241211133550900
숨 고르기 들어간 비트코인? http://v.daum.net/v/20241211142129680
비트코인 레버리지·공매도 버젓이… 당국은 `나 몰라라` http://v.daum.net/v/20241211150512600
트럼프 효과 끝?…비트코인 9만5천달러 하회 http://v.daum.net/v/20241211142219721
----------------2page 크롤링 결과----------------
비트코인 전략 제안 거부한 마이크로소프트…주주 선택은 '안정' http://v.daum.net/v/20241211085553764
[코인뉴스] "죄다 파란불이네"...비트코인 조정 국면에 알트코인도 일제히 하락 http://v.daum.net/v/20241211105754759
비트코인, 1억3600만원선까지 ‘폭락’…고점대비 10%↓ 

,title,link
0,"월가 차트분석 달인 ""비트코인 85K 추락 가능성""",http://v.daum.net/v/20241211104920355
1,'트럼프 효과 다했나'...비트코인 등 가상화폐 급락,http://v.daum.net/v/20241211232526095
2,새로운 머스크 수혜주 등장?…美대선 이후 주가 4배 껑충,http://v.daum.net/v/20241211175004287
3,가상화폐 ‘트럼프 효과’ 끝? 비트코인 10만 달러 밑으로,http://v.daum.net/v/20241211184115944
4,"코인 전문가 오태민 ""계엄 당일 우리나라만 폭락?..'리플'·'테더' 관심집중",http://v.daum.net/v/20241211191806736


## 2-3 맞춤법 검사기
- 네이버 맞춤법 검사기 이용

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [21]:
driver = webdriver.Chrome()
time.sleep(0.5)
driver.get('https://www.naver.com/')
elem = driver.find_element(By.CSS_SELECTOR, 'input#query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # enter 키

textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
textarea.send_keys('안녕하세요. 안뇽하세요. 방갑습니다')

button = driver.find_element(By.CLASS_NAME, 'btn_check')
button.click()

time.sleep(0.5)
soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.select_one('p._result_text.stand_txt').text
driver.close()
result

'안녕하세요. 안녕하세요. 반갑습니다'

### 맞춤법 검사 전 text파일을 맞춤법 검사 완료된 text파일로 output

In [22]:
# f = open('data/ch14_맞춤법검사전.txt','r',encoding='utf-8')
# text = f.read()
# f.close()

In [23]:
with open('data/ch14_맞춤법검사전.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
ready_list = []
while(len(text) > 300):
    temp = text[:300]
    last_dot_index = temp.rfind('\n')
    ready_list.append(text[:last_dot_index+1])
    print(last_dot_index)
    text = text[last_dot_index+1 : ]
ready_list.append(text[1:])

198
198
221


In [24]:
# ready_list 확인
[len(ready) for ready in ready_list]

[199, 199, 222, 296]

In [25]:
driver = webdriver.Chrome()
time.sleep(1)
driver.get('https://www.naver.com')
elem = driver.find_element(By.NAME,'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN)
time.sleep(1) # 1초 대기
# driver.implicitly_wait(1)

results = '' # 맞춤법 검사 완료된 text
textarea = driver.find_element(By.CLASS_NAME,'txt_gray')

for ready in ready_list:
    textarea.send_keys(Keys.CONTROL, 'a') # ctrl + a
    textarea.send_keys(ready)
    button = driver.find_element(By.CLASS_NAME,'btn_check')
    button.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result = soup.select_one('p._result_text').text
    results += result+'\n'
driver.close()

In [26]:
print(results)

노벨문학상은 매년 세계에서 뛰어난 문학적 업적을 이룬 작가에게 수여되는 상으로, 스웨덴의 발명가 알프레드 노벨의 유언에 따라 1901년부터 재정되었습니다. 노벨은 자신의 제산을 사용하여 인류에 공헌한 분야에 상을 주기로 하였고, 그중 하나가 바로 문학이었습니다. 매년 수상자는 스웨덴 왕이 공식 발표하며, 시상식은 이듬해 12월 스톡홀름에서 열립니다.
이 상은 특정 장르에 재한을 두지 않고, 다양한 분야에서 뛰어난 업적을 이룬 작가들에게 주어집니다. 주로 소설가, 시인, 극작가 등이 수상자로 선정되며, 문학적 기여를 인정받은 작가에게 수여됩니다. 노벨문학상은 그저 문학적 성취를 인정하는 것에 그치지 않고, 작가의 작품이 인류에 끼친 영향, 사회적 메시지, 그리고 문화적 기여 등을 평가하는 상입니다.
노벨문학상의 수상자들은 종종 사회적 문제를 다룬 작품을 통해 큰 영행을 미칩니다. 예를 들어, 알베르 카뮈는 실존주의와 부조리 문제를 다룬 작품으로, 토니 모리슨은 아프리카계 미국인의 역사와 문화를 탐구하는 작품으로 수상했습니다. 이처럼 노벨문학상은 단순히 뛰어난 문학 작품을 인정하는 것 이상으로, 작가가 창조한 작품을 통해 세계의 문제를 성찰하고 해결책을 제시하는 데 중요한 역할을 합니다.
근에는 현대 사회의 다양한 이슈들이 문학작품의 주제로 등장하면서, 노벨문학상도 그 방향에 영향을 받고 있습니다. 환경 문제, 인종 문제, 디지털 문학 등 새로운 시대적 흐름을 반영하는 작품들이 수상작으로 선정되는 추세입니다. 예를 들어, 2023년에는 일본의 소설가 오카자키 아미가 수상했으며, 그녀는 인간관계의 복잡성과 심리적 깊이를 탐구하는 작품으로 인정받았습니다.노벨문학상은 전 세계 문학계에서 가장 권위 있는 상으로, 작가들에게 큰 명예를 안겨주며, 그들의 작품이 전 세계에 더 널리 퍼질 수 있는 기회를 제공합니다.



In [27]:
# 맞춤법 결과 result를 data/ch14_맞춤법검사후.txt로 출력
with open('data/ch14_맞춤법검사후.txt','w',encoding='utf-8')as f:
    f.write(results)

# 연습문제

In [28]:
# 필요한 package import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time


# 파일 읽어오기
with open('data/ch14_맞춤법검사후.txt','r',encoding='utf-8')as f:
    text=f.read()


# 검색할 text 재단
text_list=[]
while(len(text)>1000):
    temp = text[:1000]
    last_dot_index=temp.rfind('.')
    text_list.append(temp[:last_dot_index+1])
    text=text[last_dot_index+1:]
text_list.append(text)


# 브라우저 실행
driver = webdriver.Chrome()
driver.get('https://translate.kakao.com/')


# 검색과 for문 결합
results = ''
for text in text_list:
    # 검색할 text 입력 및 버튼 실행
    elem = driver.find_element(By.CLASS_NAME, 'translate_area')
    elem.clear()
    elem.send_keys(text)
    button = driver.find_element(By.CLASS_NAME, 'btn_translate')
    button.click()
    # 검색후 결과 읽어오기
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results += soup.select_one('div.result_area').text+'\n'
    
#브라우저 종료 후 파일 출력
driver.close()
with open('data/ch14_영문번역.txt','w',encoding='utf-8') as f:
    f.write(results)

# 3절. 자동화
## 3-1. 컴퓨터가 파이썬을 실행
    - unix : corn
    - win : 작업스케쥴러
        * window + R : taskschd.msc
## 3-2. 파이썬 프로그램에서 스케쥴링
    - pip install schedule
        

In [30]:
import schedule
import time

def job():
    print('작업이 실행되었습니다.')
# job함수는 10초마다 수행할 거야
schedule.every(10).seconds.do(job)

# schedule.every().day.at('17:59').do(job)
# schedule.every().hour.do(job)
# schedule.every(1).minutes.do(job)

# 무한 루프를 돌며 스케쥴된 작업이 있는지 확인하고 자동화
while True:
    schedule.run_pending()
    time.sleep(1)

작업이 실행되었습니다.


KeyboardInterrupt: 